In [21]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
# data treatment
df = pd.read_csv('BMED_RAW_DATA.csv')
df = df.drop(['Temp_F [C]', 'Temp_A [C]', 'Temp_B [C]', 'pH_F', 'pH_A', 'pH_B'], axis=1)
df = df.dropna().reset_index(drop=True)
minLA = df[['CF_LA [M]','CA_LA [M]']].min().min()
minK = df[['CF_K [M]', 'CB_K [M]']].min().min()

In [23]:
# dataframe generation
idx = df['exp'].unique().tolist()
rdf = pd.DataFrame(columns=['V','E','t','CFLA','CALA','CFK','CBK','SEC','I','exp'])

In [24]:
i_num = [26, 27, 2, 25, 18, 5, 34, 6, 44, 17, 31, 28, 41, 40, 42]

for i in i_num:
    sdf = df[df['exp']==i][['exp','Voltage [V]','Electrolyte [M]','Time [hr]','Ci_LA [M]','Ci_K [M]','CF_LA [M]','CA_LA [M]','CF_K [M]','CB_K [M]','VF_Tank [L]','VA_Tank [L]','VB_Tank [L]','VF_Line [L]','VA_Line [L]','VB_Line [L]','Current [A]']]

    cal = pd.DataFrame()
    cal['time'] = sdf['Time [hr]']
    cal['VF'] = sdf['VF_Tank [L]'] + sdf['VF_Line [L]']
    cal['VA'] = sdf['VA_Tank [L]'] + sdf['VA_Line [L]']
    cal['VB'] = sdf['VB_Tank [L]'] + sdf['VB_Line [L]']
    cal['TVi'] = cal['VF'] + cal['VA'] + cal['VB']
    cal['VFm'] = cal['VF'] * cal['TVi'].iloc[0] / cal['TVi']
    cal['VAm'] = cal['VA'] * cal['TVi'].iloc[0] / cal['TVi']
    cal['VBm'] = cal['VB'] * cal['TVi'].iloc[0] / cal['TVi']
    cal['TVm'] = cal['VFm'] + cal['VAm'] + cal['VBm']

    cal['CFi_LA'] = sdf['CF_LA [M]'] - minLA
    cal['CAi_LA'] = sdf['CA_LA [M]'] - minLA

    cal['NFi_LA'] = cal['CFi_LA'] * cal['VFm']
    cal['NAi_LA'] = cal['CAi_LA'] * cal['VAm']
    cal['TNi_LA'] = cal['NFi_LA'] + cal['NAi_LA']
    cal['NFm_LA'] = cal['NFi_LA'] * cal['TNi_LA'].iloc[0] / cal['TNi_LA']
    cal['NAm_LA'] = cal['NAi_LA'] * cal['TNi_LA'].iloc[0] / cal['TNi_LA']
    cal['TNm_LA'] = cal['NFm_LA'] + cal['NAm_LA']

    cal['CFi_K'] = sdf['CF_K [M]'] - minK
    cal['CBi_K'] = sdf['CB_K [M]'] - minK

    cal['NFi_K'] = cal['CFi_K'] * cal['VFm']
    cal['NBi_K'] = cal['CBi_K'] * cal['VBm']
    cal['TNi_K'] = cal['NFi_K'] + cal['NBi_K']
    cal['NFm_K'] = cal['NFi_K'] * cal['TNi_K'].iloc[0] / cal['TNi_K']
    cal['NBm_K'] = cal['NBi_K'] * cal['TNi_K'].iloc[0] / cal['TNi_K']
    cal['TNm_K'] = cal['NFm_K'] + cal['NBm_K']

    cal['CFm_LA'] = cal['NFm_LA'] / cal['VFm']
    cal['CAm_LA'] = cal['NAm_LA'] / cal['VAm']
    cal['CFm_K'] = cal['NFm_K'] / cal['VFm']
    cal['CBm_K'] = cal['NBm_K'] / cal['VBm']

    sdf_s = pd.DataFrame()
    sdf_s['V'] = sdf['Voltage [V]']; sdf_s['E'] = sdf['Electrolyte [M]']; sdf_s['time'] = cal['time'];
    sdf_s['CFLA'] = cal['CFm_LA']; sdf_s['CALA'] = cal['CAm_LA']; 
    sdf_s['CFK'] = cal['CFm_K']; sdf_s['CBK'] = cal['CBm_K'];
    sdf_s['VF'] = cal['VFm']; sdf_s['VA'] = cal['VAm']; sdf_s['VB'] = cal['VBm'];
    sdf_s['I'] = sdf['Current [A]']; sdf_s['exp'] = i
    sdf_s = sdf_s.reset_index(drop=True)

    rdf = pd.concat([rdf, sdf_s], ignore_index=True)

/tmp/ipykernel_2172/1999487822.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  rdf = pd.concat([rdf, sdf_s], ignore_index=True)


In [25]:
rdf.to_csv('BMED_DATA.csv', index=False)